# Generador de personas aleatorias

In [85]:
import random as rm
import numpy as np
import pandas as pd
from faker import Faker
import datetime as dt
import pytz
import re
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
from funciones import quitar_acentos
import urllib.parse
from typing import Union

In [168]:
def sacar_curp(nombre: str, apellido_paterno: str, apellido_materno: str, sexo: str, fecha_nac: Union[str, dt.datetime], estado_nac: str) -> str:
	nombre = quitar_acentos(nombre).upper().replace('Ñ', 'X')
	apellido_paterno = quitar_acentos(apellido_paterno).upper().replace('Ñ', 'X')
	apellido_materno = quitar_acentos(apellido_materno).upper().replace('Ñ', 'X')
	estado_nac = quitar_acentos(estado_nac)
	if isinstance(fecha_nac, str): fecha_nac = dt.datetime.strptime(fecha_nac, '%Y-%m-%d')
	# Primera letra y vocal del primer apellido
	curp = apellido_paterno[0:2].upper()
	# Primera letra del segundo apellido
	curp += apellido_materno[0:1].upper()
	# Primera letra del nombre de pila
	curp += nombre[0:1].upper()
	# Fecha de nacimiento (2 últimos dígitos del año, 2 del mes y 2 del día de nacimiento)
	curp += fecha_nac.strftime('%y%m%d')
	# Letra del sexo (H o M)
	curp += sexo[0:1].upper()
	# Dos letras correspondientes a la entidad de nacimiento (Sonora => SR)
	# en el caso de extranjeros, se marca como NE (Nacido Extranjero)
	clave_estados = {
		'aguascalientes': 'AS',
		'baja_california': 'BC',
		'baja_california_sur': 'BS',
		'campeche': 'CC',
		'coahuila': 'CL',
		'colima': 'CM',
		'chiapas': 'CS',
		'chihuahua': 'CH',
		'distrito_federal': 'DF',
		'ciudad_de_mexico': 'DF',
		'durango': 'DG',
		'guanajuato': 'GT',
		'guerrero': 'GR',
		'hidalgo': 'HG',
		'jalisco': 'JC',
		'mexico': 'MC',
		'estado_de_mexico': 'MC',
		'michoacan': 'MN',
		'morelos': 'MS',
		'nayarit': 'NT',
		'nuevo_leon': 'NL',
		'oaxaca': 'OC',
		'puebla': 'PL',
		'queretaro': 'QT',
		'quintana_roo': 'QR',
		'san_luis_potosi': 'SP',
		'sinaloa': 'SL',
		'sonora': 'SR',
		'tabasco': 'TC',
		'tamaulipas': 'TS',
		'tlaxcala': 'TL',
		'veracruz': 'VZ',
		'yucatan': 'YN',
		'zacatecas': 'ZS'
	}
	estado_nac_key = estado_nac.replace(' ', '_').lower()
	if estado_nac_key in clave_estados:
		clave_estado_nac = clave_estados[estado_nac_key]
	else:
		clave_estado_nac = 'NE'
	curp += clave_estado_nac
	# Primera consonante interna del primer apellido
	curp += re.sub(f'[aeiou]', '', apellido_paterno)[1:2].upper()
	# Primera consonante interna del segundo apellido
	curp += re.sub(f'[aeiou]', '', apellido_materno)[1:2].upper()
	# Primera consonante interna del nombre
	curp += re.sub(f'[aeiou]', '', nombre)[1:2].upper()
	# Dígito verificador del 0-9 para fechas de nacimiento hasta el año 1999 y A-Z para fechas de nacimiento a partir del 2000
	int_char = [rm.randint(0, 9), chr(rm.randint(65, 90))]
	curp += str(int_char[0] if fecha_nac.year <= 1999 else int_char[1])
	# Homoclave, para evitar duplicaciones
	curp += str(rm.randint(0, 9))
	return curp

def generar_puntos_random(ciudad, estado, pais = 'mexico', n = 1, reducir = True):
	# hacer peticion a un api para obtener coordenadas del centro y extremas
	querystring = urllib.parse.urlencode({'city': ciudad, 'state': estado, 'country': pais})
	df = pd.read_json(f'https://nominatim.openstreetmap.org/search?{querystring}&polygon_geojson=1&format=json')
	# obtener coordenadas del centro y extremos
	cenx, ceny = (df['lat'][0], df['lon'][0])
	minx, maxx, miny, maxy = [float(i) for i in df['boundingbox'][0]]
	# obtener extremos originales
	ne, no, se, so = [(maxx, maxy), (maxx, miny), (minx, maxy), (minx, miny)]

	# hallar punto medio entre (suroeste centro) y (centro noreste)
	minx, miny = ((minx + cenx) / 2), ((miny + ceny) / 2)
	maxx, maxy = ((cenx + maxx) / 2), ((ceny + maxy) / 2)

	print(f"Coordenadas de {ciudad} {estado} {pais}")
	# true para reducir el area de la ciudad
	if reducir:
		# hallar punto medio de puntos medios y centro
		minx, miny = ((minx + cenx) / 2), ((miny + ceny) / 2)
		maxx, maxy = ((cenx + maxx) / 2), ((ceny + maxy) / 2)

		print('centro:', (cenx, ceny))
		print('noreste reducido:', (maxx, maxy))
		print('noroeste reducido:', (maxx, miny))
		print('sureste reducido:', (minx, maxy))
		print('suroeste reducido:', (minx, miny))
	else:
		print('centro:', (cenx, ceny))
		print('noreste:', ne)
		print('noroeste:', no)
		print('sureste:', se)
		print('suroeste:', so)

	# obtener n puntos aleatorios dentro del area
	puntos_random = []
	if n > 1:
		while len(puntos_random) < n:
			puntos_random.append((rm.uniform(minx, maxx), rm.uniform(miny, maxy)))
	else:
		puntos_random = (rm.uniform(minx, maxx), rm.uniform(miny, maxy))
	print('random:', puntos_random)
	return puntos_random

geolocator = Nominatim(user_agent = 'geoapiExercises')
punto_random = generar_puntos_random('hermosillo', 'sonora')
listo = False
while not listo:
	direccion = geolocator.reverse(punto_random, language = 'es_MX')
	direccion = direccion[0].split(', ')
	print(direccion)
	print(len(direccion))
	dirr = {}
	if len(direccion) == 8:
		dirr['local'] = direccion[0]
		dirr['numero'] = direccion[1]
		dirr['calle'] = direccion[2]
		dirr['colonia'] = direccion[3]
		dirr['cp'] = direccion[-2]
		print(', '.join(str(i) for i in dirr.values()))
		listo = True
	elif len(direccion) == 7:
		dirr['local'] = direccion[0]
		dirr['calle'] = direccion[1]
		dirr['numero'] = rm.randint(1, 999)
		dirr['colonia'] = direccion[2]
		dirr['cp'] = direccion[-2]
		print(', '.join(str(i) for i in dirr.values()))
		listo = True
	elif len(direccion) == 6:
		dirr['calle'] = direccion[0]
		dirr['numero'] = rm.randint(1, 999)
		dirr['colonia'] = direccion[1]
		dirr['cp'] = direccion[-2]
		print(', '.join(str(i) for i in dirr.values()))
		listo = True
	else:
		print('no se encontro calle o colonia')
	# quitar pais cp estado ciudad
	del direccion[-4:]
	print(direccion)

Coordenadas de hermosillo sonora mexico
centro: (29.0948207, -110.9692202)
noreste reducido: (29.1348207, -110.9292202)
noroeste reducido: (29.1348207, -111.0092202)
sureste reducido: (29.0548207, -110.9292202)
suroeste reducido: (29.0548207, -111.0092202)
random: (29.1073115558528, -110.93124895873032)
['Villa de los Zafiros', 'Hermosillo', 'Sonora', '83160', 'México']
5
no se encontro calle o colonia
['Villa de los Zafiros']
['Villa de los Zafiros', 'Hermosillo', 'Sonora', '83160', 'México']
5
no se encontro calle o colonia
['Villa de los Zafiros']
['Villa de los Zafiros', 'Hermosillo', 'Sonora', '83160', 'México']
5
no se encontro calle o colonia
['Villa de los Zafiros']
['Villa de los Zafiros', 'Hermosillo', 'Sonora', '83160', 'México']
5
no se encontro calle o colonia
['Villa de los Zafiros']
['Villa de los Zafiros', 'Hermosillo', 'Sonora', '83160', 'México']
5
no se encontro calle o colonia
['Villa de los Zafiros']
['Villa de los Zafiros', 'Hermosillo', 'Sonora', '83160', 'México

KeyboardInterrupt: 

In [101]:
reap = pd.read_csv('./conjunto_datos/relacion_edad_altura_peso.csv')
nombres = pd.read_csv('./catalogos/nombres.csv')
profesiones = pd.read_csv('./catalogos/profesiones.csv')
file_ciudades = './catalogos/ciudades.csv'
ct_ciudades = pd.read_csv(file_ciudades)

def personas_aleatorias(n = 5):
	fake = Faker(['es_MX'])
	personas = []
	for _ in range(n):
		sexo = rm.choice(['H', 'M'])
		# generar tipo de sangre y asignar probabilidad a cada valor para que salga
		tipo_sangre = rm.choices(['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-'], weights = [30, 6, 9, 2, 4, 1, 39, 9])
		tipo_sangre = tipo_sangre[0]

		fecha_nac = fake.date_of_birth(minimum_age = int(reap['edad'].min()))
		# calcular edad en base a la fecha de nacimiento
		dia_actual = dt.date.today()
		edad = dia_actual.year - fecha_nac.year - ((dia_actual.month, dia_actual.day) < (fecha_nac.month, fecha_nac.day))

		# dependiendo de la edad se busca el margen de alturas y pesos que tenga esa edad
		# si es mayor que la edad maxima que este registrada se le da el margen de esa edad maxima
		# mucha gente deja de crecer a los 20
		margen_ap = reap[(reap['edad'] == edad) if edad <= reap['edad'].max() else (reap['edad'] == reap['edad'].max())].reset_index()

		# dependiendo del sexo se generan nombres femeninos o masculinos
		# y estaturas y pesos dentro de su margen correspondiente
		if sexo == 'H':
			titulo = rm.choice(['Sr.', 'Dr.', 'Mtro.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['h'])
			altura = round(rm.uniform(margen_ap['lim_inf.altura.h.cm'][0], margen_ap['lim_sup.altura.h.cm'][0]), 2)
			peso = round(rm.uniform(margen_ap['lim_inf.peso.h.kg'][0], margen_ap['lim_sup.peso.h.kg'][0]), 2)
		else:
			titulo = rm.choice(['Sra.', 'Dra.', 'Mtra.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['m'])
			altura = round(rm.uniform(margen_ap['lim_inf.altura.m.cm'][0], margen_ap['lim_sup.altura.m.cm'][0]), 2)
			peso = round(rm.uniform(margen_ap['lim_inf.peso.m.kg'][0], margen_ap['lim_sup.peso.m.kg'][0]), 2)

		apellido_paterno = fake.last_name()
		apellido_materno = fake.last_name()

		id_estado = rm.choice(ct_ciudades['id_estado'].unique())
		ct_ciudad = ct_ciudades[ct_ciudades['id_estado'] == id_estado].sample()

		ciudad = ct_ciudad['ciudad'].values[0]
		estado = ct_ciudad['estado'].values[0]

		tel = f'({ct_ciudad["clave_lada"].values[0]})' + str(rm.randint(1000000, 9999999))

		curp = sacar_curp(nombre, apellido_paterno, apellido_materno, sexo, fecha_nac, estado)

		# Generar un nombre de usuario para un email
		switcher = {
			1: fake.user_name(),
			# aperez
			2: f'{nombre[0]}{apellido_paterno}',
			# alonso_perez
			3: f'{nombre}_{apellido_paterno}',
			# aps70
			4: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.strftime("%y")}',
			# aps1970
			5: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.year}',
			# alonsoperezsoltero
			6: f'{nombre}{apellido_paterno}{apellido_materno}'
		}
		rand_int = rm.randint(min(list(switcher.keys())), max(list(switcher.keys())))
		usuario = switcher.get(rand_int, 'usuario')
		# quitar espacios o reemplazarlos por un _
		usuario = usuario.replace(' ', rm.choice(['', '_'])).lower()
		usuario = quitar_acentos(usuario)
		subdominio = rm.choice(['.com', '.net', '.io', '.gob', '.org', '.edu'])
		correo = usuario + f'@example{subdominio}'

		direccion = fake.street_address()
		direcc_calle = fake.street_name()
		direcc_numero = rm.randint(1, 999)
		direcc_cp = fake.postcode()

		profesion = profesiones.sample().values[0][0] if edad >= 18 else None
		"""
		El Número de Seguridad Social (NSS) es una homoclave que asigna el IMSS
		para llevar un registro de los trabajadores y asegurados
		que están inscritos en dicha institución.
		El NSS es único, permanente e intransferible.
		"""
		# El NSS está compuesto por 11 dígitos:
		if profesion:
			# Los primeros dos caracteres están vinculados a la subdelegación en el que la persona fue afiliada.
			nss1 = ('%02d' % rm.randint(0, 99))
			# Los dos dígitos siguientes indican el año en el que la persona se afilió al Seguro Social.
			nss2 = ('%02d' % rm.randint(0, 99))
			# Los siguientes dos dígitos corresponden a la fecha de nacimiento del afiliado.
			nss3 = fecha_nac.strftime('%y')
			# Los cuatro números siguientes son los dígitos que asigna el IMSS al trabajador.
			nss4 = ('%02d' % rm.randint(0, 9999))
			# El último dígito corresponde al número de verificación del trabajador en el IMSS.
			nss5 = str(rm.randint(0, 9))
			nss = f'{nss1} {nss2} {nss3} {nss4} {nss5}'
		else:
			nss = None

		"""
		Latitud y longitud de la ciudad
		"""
		# si el catalogo ya tiene coordenadas puestas, tomar esos valores
		if pd.notna(ct_ciudad['latitud'].values[0]) and pd.notna(ct_ciudad['longitud'].values[0]):
			lat = ct_ciudad['latitud'].values[0]
			lon = ct_ciudad['longitud'].values[0]
		else:
			# si no, buscar en internet la latitud y longitud segun la ciudad
			try:
				geolocator = Nominatim(user_agent = 'geoapiExercises')
				location = geolocator.geocode(f'{ciudad}, {estado}, Mexico', language = 'es_MX')
				if location:
					lat = location.latitude
					lon = location.longitude

					# guardar coordenadas en el catalogo
					# para que a la otra no haya que buscar en internet
					ct_ciudad_index_row = ct_ciudad.index.values[0]
					ct_ciudades.loc[ct_ciudad_index_row, 'latitud'] = lat
					ct_ciudades.loc[ct_ciudad_index_row, 'longitud'] = lon
					ct_ciudades.to_csv(file_ciudades, index = False)
				else:
					raise BaseException(f'(idx {_}) No se encontro latitud ni longitud de {ciudad}, {estado}')
			except BaseException as be:
				# En caso de no haber encontrado o no tener internet
				print('\033[31mException:\033[0m')
				print(be)
				lat = lon = np.NaN

		"""
		Zona horaria
		"""
		if not np.isnan(lat) and not np.isnan(lon):
			# obtener zona horaria segun latitud y longitud
			obj = TimezoneFinder()
			timezone = obj.timezone_at(lat = lat, lng = lon)
			# 'datetime.now' tiene DST (Daylight Saving Time)
			tz_offset = dt.datetime.now(pytz.timezone(timezone)).strftime('%z')
		# en Enero no hay DST
		#tz_offset = pytz.timezone(timezone).localize(dt.datetime(2011, 1, 1)).strftime('%z')
		else:
			timezone = tz_offset = None

		personas.append({
			'titulo': titulo,
			'nombre': nombre,
			'apellido_paterno': apellido_paterno,
			'apellido_materno': apellido_materno,
			'fecha_nac': fecha_nac.strftime('%Y-%m-%d'),
			'sexo': sexo,
			'altura': altura,
			'peso': peso,
			'tipo_sangre': tipo_sangre,
			'correo': correo,
			'tel': tel,
			# 'calle': '',
			# 'numero': '',
			# 'cp': '',
			'ciudad': ciudad,
			'estado': estado,
			'lat': lat,
			'lon': lon,
			'tz_offset': tz_offset,
			'zona_horaria': timezone,
			'profesion': profesion,
			'curp': curp,
			'nss': nss,
			# 'color_fav': fake.safe_color_name(),
		})
	return pd.DataFrame(personas)

personas_aleatorias(10)

Exception:
HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Tlapanaloya%2C+Estado+de+Mexico%2C+Mexico&format=json&limit=1&accept-language=es_MX (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Exception:
(idx 1) No se encontro latitud ni longitud de San Gabriel  Docuan, Estado de Mexico


,titulo,nombre,apellido_paterno,apellido_materno,fecha_nac,sexo,altura,peso,tipo_sangre,correo,tel,ciudad,estado,lat,lon,tz_offset,zona_horaria,profesion,curp,nss
0,None,Felipe,Borrego,Gracia,1973-03-05,H,183.07,79.10,O-,poncepilar@example.gob,(599)7509973,Tlapanaloya,Estado de Mexico,NaN,NaN,None,None,Científico biomédico,BOGF730305HMCORE64,48 96 73 3765 1
1,None,Guadalupe,Acevedo,Colón,1965-03-06,M,162.51,75.58,O-,gacevedo@example.edu,(588)7062451,San Gabriel Docuan,Estado de Mexico,NaN,NaN,None,None,Logística/apoyo/oficial administrativo de las ...,ACCG650306MMCCOU03,07 12 65 210 3
2,None,Alberto,Villanueva,Santacruz,2012-07-28,H,141.00,28.45,A+,albertovillanuevasantacruz@example.net,(786)5233419,Senguio,Michoacan,19.746456,-100.347360,-0600,America/Mexico_City,None,VISA120728HMNIALF2,None
3,None,Nelly,Aparicio,Gaona,1978-10-31,M,153.36,71.77,AB+,nag1978@example.com,(826)5212573,General Terán,Nuevo Leon,25.270658,-99.408929,-0600,America/Monterrey,Bioquímico clínico,APGN781031MNLPAE42,49 20 78 3717 8
4,None,Gabriela,Zúñiga,Carrión,1985-12-01,M,163.63,82.70,O-,gzuniga@example.net,(731)3105544,Huecahuasco,Morelos,18.937993,-98.781925,-0600,America/Mexico_City,Guardabosques,ZUCG851201MMSUAA90,17 64 85 2684 5
5,None,Carolina,Carrasco,Altamirano,1918-08-05,M,174.65,80.11,O-,carolinacarrascoaltamirano@example.io,(774)5196793,Cerro Colorado,Hidalgo,20.064769,-98.952911,-0600,America/Mexico_City,Oficial de entrenamiento y educación de las fu...,CAAC180805MHGALA82,75 78 18 448 2
6,Dra.,María Teresa,Saavedra,Tijerina,1998-09-17,M,154.96,50.96,O+,mst98@example.gob,(248)2486807,San Martín Texmelucan,Puebla,19.265984,-98.424878,-0600,America/Mexico_City,Librero,SATM980917MPLAIA99,71 44 98 7298 4
7,Dra.,Anabel,Prado,Domínguez,1999-10-30,M,158.26,45.95,B+,aprado@example.edu,(996)2890324,Hecelchakan,Campeche,20.177437,-90.133885,-0600,America/Merida,Administrador de organizaciones benéficas/orga...,PRDA991030MCCRON97,88 57 99 5883 4
8,None,Eugenio,Orozco,Cepeda,1943-06-15,H,177.75,78.99,O-,eugenio_orozco@example.com,(917)9780046,Huimanguillo,Tabasco,17.762097,-93.663441,-0600,America/Mexico_City,Tecnólogo de ropa/textil,ORCE430615HTCREU53,38 00 43 5464 3
9,Dra.,Ilse,Hernandes,Durán,1998-10-18,M,165.74,67.80,O-,ihernandes@example.io,(448)4876974,"La ""d""",Queretaro,20.440231,-100.153616,-0600,America/Mexico_City,Oficial de caridad,HEDI981018MQTEUL43,64 40 98 2319 9


In [29]:
# mas info en https://randomuser.me/documentation
url_api = 'https://randomuser.me/api?format=csv'
params = {
	'results': 100,
	#'nat': 'es',
	#'password': '8-16,lower,upper,number,special',
	'exc': 'nat,picture,id,login,registered'
}
query = ''
for k, v in params.items():
	if isinstance(v, int): v = str(v)
	query += '&' + k + '=' + v
url = url_api + query
people = pd.read_csv(url)
people.sample(8)

,gender,name.title,name.first,name.last,location.street.number,location.street.name,location.city,location.state,location.country,location.postcode,location.coordinates.latitude,location.coordinates.longitude,location.timezone.offset,location.timezone.description,email,dob.date,dob.age,phone,cell
33,female,Miss,Shenna,Van Ravenhorst,2915,Halteriastraat,Scheulder,Drenthe,Netherlands,17500,-15.8411,165.5078,-5:00,"Eastern Time (US & Canada), Bogota, Lima",shenna.vanravenhorst@example.com,1982-01-15T14:04:18.916Z,40,(026)-467-5304,(525)-688-0957
78,male,Mr,Mason,White,150,Wiri Station Road,Palmerston North,Wellington,New Zealand,84303,-84.5117,-152.2561,+10:00,"Eastern Australia, Guam, Vladivostok",mason.white@example.com,1984-01-22T08:59:50.427Z,38,(278)-670-2846,(137)-909-5200
71,female,Mrs,Inger,Aam,4256,Ørakerfaret,Slemsrud,Description,Norway,6871,-78.4872,121.0481,+1:00,"Brussels, Copenhagen, Madrid, Paris",inger.aam@example.com,1969-10-19T08:58:50.214Z,53,67000514,96911015
45,male,Mr,علی,سالاری,6913,جمهوری اسلامی,نجف‌آباد,هرمزگان,Iran,78077,-75.0953,69.0367,-11:00,"Midway Island, Samoa",aaly.slry@example.com,1986-07-18T07:21:40.800Z,36,062-02282599,0928-131-1938
55,female,Ms,Nanna,Petersen,7113,Stengårdsvej,Randers Nv,Midtjylland,Denmark,57406,36.0369,36.7946,+8:00,"Beijing, Perth, Singapore, Hong Kong",nanna.petersen@example.com,1984-10-01T14:22:00.382Z,38,95755048,44487998
24,male,Mr,Felix,Anderson,6654,Lake of Bays Road,Enterprise,Québec,Canada,J0C 8E1,5.9558,-179.8199,+2:00,"Kaliningrad, South Africa",felix.anderson@example.com,1984-04-07T15:14:45.550Z,38,937-488-7448,190-416-3300
83,female,Mrs,Blanca,Martin,2146,Calle de Ángel García,Madrid,Asturias,Spain,53533,-55.8710,-140.4980,-6:00,"Central Time (US & Canada), Mexico City",blanca.martin@example.com,1991-06-30T15:53:12.237Z,31,982-352-967,678-111-568
76,male,Mr,Óscar,da Rosa,3272,Rua Santa Rita,Barretos,Goiás,Brazil,92501,75.1005,-130.2087,+5:30,"Bombay, Calcutta, Madras, New Delhi",oscar.darosa@example.com,1983-06-05T14:34:37.349Z,39,(44) 6825-3100,(90) 8831-5559
